In [4]:
import sys
import os
# LIBRARY_PATH = '/home/deleidos/Notebooks/Newspaper/SemanticEstimation'
LIBRARY_PATH = "/home/deleidos/Notebooks/lib"
module_path = os.path.abspath(os.path.join(LIBRARY_PATH))
if module_path not in sys.path:
    sys.path.append(module_path)
print (f"sys.path : {sys.path}") 

sys.path : ['/home/deleidos/Notebooks/Newspaper/SemanticEstimation', '/opt/conda/lib/python38.zip', '/opt/conda/lib/python3.8', '/opt/conda/lib/python3.8/lib-dynload', '', '/opt/conda/lib/python3.8/site-packages', '/opt/conda/lib/python3.8/site-packages/IPython/extensions', '/home/deleidos/.ipython', '/home/deleidos/Notebooks/lib']


In [7]:
import newspaper
from newspaper import Article 
from keybert import KeyBERT

import json 
import boto3 
import logging 
import uuid 
import time
from datetime import date, datetime 
import datetime
from pathlib import Path 
import os 
import glob
import nltk
# from bertopic import BERTopic
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer

from transformers import pipeline
import pandas as pd
import contractions

# nltk.download('punkt')
# setup widget import
from ipywidgets import widgets,Layout
from IPython.display import display

# !pip install pyldavis
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from pprint import pprint

# import spacy
# import pickle
# import re
# import pyLDAvis
# import pyLDAvis.gensim_models

#LOCALS
RESOURCE = 'dynamodb'
TABLE = "Articles-tfoo6wfq2zbxfdwl3z47nsgtrq-dev"
REGION = "us-east-1"

CSVPath = "/home/deleidos/Notebooks/CSVData"
JSONPath = "/home/deleidos/Notebooks/JSONData"
MODELPATH= "/home/deleidos/Notebooks/Models"

from dynamoDButils import *
from APIutils import *
from seedterms import *


def complete(state="complete"):
    print(f"\nCell {state} @ {(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))}")
complete(state='imports done')


Cell imports done @ 2023-03-04 14:02:20


[nltk_data] Downloading package punkt to /home/deleidos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
# from transformers import pipeline
classifier = pipeline("zero-shot-classification"
                    ,model="facebook/bart-large-mnli" 
                    ,device=0                   
                     ) # to utilize GPU specify device= ...

complete(state='classifier initialised')


Cell classifier initialised @ 2023-03-04 14:02:33


In [9]:
candidate_labels = ['technology', 'innovation','politics', 'electronics','entertainment', 'economics'
                    , 'advertisement', 'social', 'games'
                    , 'sports', 'health', 'science'
                    , 'business innovation']
df = pd.DataFrame()
articles_csv = []

In [10]:
jsonfiles = [f for f in listdir(JSONPath) if isfile(join(JSONPath, f))]
jsonfiles.sort()
jsonfiles = [x for x in jsonfiles if (os.path.splitext(x)[1][1:]).lower() == "json"]
jsnWD = widgets.Select(
    options=jsonfiles,
    value=jsonfiles[0],
    description='Choose ID:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        layout=Layout(width='50%', height='256px')
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
#         print ("changed to %s" % change['new'])
        selection = change['new']

jsnWD.observe(on_change)
display(jsnWD)

Select(description='Choose ID:', layout=Layout(height='256px', width='50%'), options=('2000CWE.json', 'NOTsema…

In [11]:
# iWD.value
jsnf = jsnWD.value
print (f"{jsnf=} ")

jsnf='semanticSimArticlesTestingDF.json' 


#### Read Candidate articles s/t we can compare to the model
Here, we begin the process of looking at a collection of articles and rating them for inclusion vice a model constructed from the existing corpus of articles

In [12]:
jsnfile = osp.join (JSONPath,jsnf)
summaries = pd.read_json(jsnfile)
# xposedf = df.transpose() # pivot the matrix

In [13]:
pd.set_option("max_colwidth", 30)
pd.set_option("expand_frame_repr", True)
summaries

,article,summary,hf1,hf2,hf3,hf4,hf5,hfAvg,kb1,kb2,kb3,kb4,kb5,kbAvg,topic1,score1,topic2,score2
0,https://datainnovation.org...,economist created visualiz...,0.5627,0.5556,0.5153,0.5132,0.4939,0.528140,0.408926,0.408615,0.395756,0.385963,0.365358,0.392923,economics,0.629220,politics,0.167246
1,https://venturebeat.com/ai...,among organization buildin...,0.9986,0.6797,0.6685,0.6572,0.6319,0.659325,0.515490,0.456996,0.455971,0.445295,0.442166,0.450107,technology,0.585379,innovation,0.180912
2,https://venturebeat.com/ai...,watch demand session today...,0.5767,0.5576,0.5529,0.5504,0.5455,0.551600,0.549421,0.546504,0.544978,0.534699,0.533749,0.541870,entertainment,0.176518,social,0.170545
3,https://venturebeat.com/ai...,generative going completel...,0.7521,0.7365,0.7272,0.7240,0.7216,0.732280,0.519292,0.496403,0.488997,0.482039,0.477190,0.486157,innovation,0.602014,business innovation,0.258873
4,https://www.businessinside...,personalized cancer vaccin...,0.5688,0.5227,0.4532,0.4527,0.4452,0.488520,0.428122,0.427320,0.422924,0.419798,0.395541,0.424541,innovation,0.430343,technology,0.140970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,https://www.cbsnews.com/ne...,spacex falcon rocket carry...,0.8183,0.4993,0.4775,0.4735,0.4719,0.480550,0.531412,0.511708,0.454913,0.436846,0.434590,0.473894,technology,0.496222,innovation,0.111382
121,https://www.cbsnews.com/vi...,rise could chatgpt take jo...,0.7697,0.7255,0.7166,0.7103,0.6910,0.710850,0.637201,0.625964,0.625526,0.619846,0.619757,0.622773,technology,0.675250,innovation,0.216986
122,https://www.cbsnews.com/ne...,jonas ohman founder ceo bl...,0.4923,0.4892,0.4892,0.4847,0.4596,0.488850,0.457436,0.457436,0.430608,0.427529,0.421351,0.438872,technology,0.349854,social,0.111065
123,https://www.cbsnews.com/ne...,metaverse asset digital ar...,0.6600,0.6304,0.6264,0.6229,0.6044,0.626567,0.614686,0.609527,0.597931,0.587996,0.571264,0.596281,games,0.463839,entertainment,0.240798


In [14]:
# remove protocols
summaries.summary = summaries.apply(lambda row: re.sub(r"http\S+", "", row.summary).lower(), 1)
# remove non-letters
summaries.summary = summaries.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.summary).split()), 1)
# convert to lowercase
summaries.summary = summaries.summary.apply(lambda x: ' '.join([w.lower() for w in x.split()]))
# expand contractions  
summaries.summary = summaries.summary.apply(lambda x: ' '.join([contractions.fix(w) for w in x.split()]))
# remove stopwords
stop_words = stopwords.words('english')
summaries.summary = summaries.summary.apply(lambda x: ' '.join([w for w in x.split() if w not in stop_words]))
# remove short words
summaries.summary = summaries.summary.apply(lambda x: ' '.join([w.strip() for w in x.split() if len(w.strip()) >= 3]))
# lemmatize
summaries.summary = summaries.summary.apply(lambda x: ' '.join([WordNetLemmatizer().lemmatize(w) for w in x.split()]))
# docs = summaries.summary.to_list()
# urls = summaries.article.to_list()
# print (f"Number of articles for training the model: {len(docs)}")    
docs = []
for index, row in summaries.iterrows():
    docs.append([row['article'], row['summary']])
#     print(row['article'], row['summary']) 

complete(state="models")
docs[0]


Cell models @ 2023-03-04 14:02:50


['https://datainnovation.org/2022/11/visualizing-changes-to-politicians-followers-on-twitter/',
 'economist created visualization showing politician twitter follower changed elon musk acquired platform visualization show percent change individual account party average october october according visualization republican gained around follower musk acquired twitter democrat lost follower take look']

In [15]:
def load_candidate_articles(docs=None, candidate_labels=None):
    for doc in docs:
        url = doc[0]
        text = doc[1]
        classify = classifier(text, candidate_labels)
        articles_csv.append({'url': url,
                 'text': text,
                 'topic1': classify['labels'][0],
                 'score1': classify['scores'][0],
                 'topic2': classify['labels'][1],
                 'score2': classify['scores'][1],                                         
                }) 
    
    df = pd.DataFrame(articles_csv)
#     df.to_csv('articles20220919.csv')
    return df

In [16]:
%%time
dfClassifier = load_candidate_articles(docs=docs, candidate_labels=candidate_labels)

CPU times: user 1min 11s, sys: 9.89 s, total: 1min 21s
Wall time: 1min 21s


In [14]:
dfClassifier

,url,text,topic1,score1,topic2,score2
0,https://datainnovation.org...,economist created visualiz...,economics,0.629220,politics,0.167246
1,https://venturebeat.com/ai...,among organization buildin...,technology,0.585379,innovation,0.180912
2,https://venturebeat.com/ai...,watch demand session today...,entertainment,0.176518,social,0.170545
3,https://venturebeat.com/ai...,generative going completel...,innovation,0.602014,business innovation,0.258873
4,https://www.businessinside...,personalized cancer vaccin...,innovation,0.430343,technology,0.140970
...,...,...,...,...,...,...
120,https://www.cbsnews.com/ne...,spacex falcon rocket carry...,technology,0.496222,innovation,0.111382
121,https://www.cbsnews.com/vi...,rise could chatgpt take jo...,technology,0.675250,innovation,0.216986
122,https://www.cbsnews.com/ne...,jonas ohman founder ceo bl...,technology,0.349854,social,0.111065
123,https://www.cbsnews.com/ne...,metaverse asset digital ar...,games,0.463839,entertainment,0.240798


In [15]:
TECHLABEL='technology'
INNOLABEL='innovation'
BUSILABEL='business innovation'
SCIXLABEL= 'science'
newdf = dfClassifier.loc[
    (
        (
        ((dfClassifier['topic1']==TECHLABEL) & (dfClassifier['topic2']==INNOLABEL))  |
        ((dfClassifier['topic1']==TECHLABEL) & (dfClassifier['topic2']==BUSILABEL))  |
        ((dfClassifier['topic1']==TECHLABEL) & (dfClassifier['topic2']==SCIXLABEL))  |
        ((dfClassifier['topic1']==BUSILABEL) & (dfClassifier['topic2']==TECHLABEL))  |
        ((dfClassifier['topic1']==SCIXLABEL) & (dfClassifier['topic2']==TECHLABEL))  |        
        ((dfClassifier['topic1']==INNOLABEL) & (dfClassifier['topic2']==TECHLABEL) )) &
        (dfClassifier['score1'] + dfClassifier['score2'] > 0.5)
        ) 
    , ['url','text','topic1','score1','topic2','score2']
]

In [16]:
len(newdf)

26

In [17]:
newdf.head()

,url,text,topic1,score1,topic2,score2
1,https://venturebeat.com/ai...,among organization buildin...,technology,0.585379,innovation,0.180912
4,https://www.businessinside...,personalized cancer vaccin...,innovation,0.430343,technology,0.140970
11,https://www.govpilot.com/g...,property level detail data...,technology,0.545680,innovation,0.186994
12,https://www.techrepublic.c...,azure industry solution cu...,technology,0.414966,business innovation,0.194544
13,https://www.technology-in-...,quantum computer making cl...,technology,0.598974,innovation,0.267198


### Now we need to add these to our JSON file

Instead of combining the secondary scoring results (newdf) with our master DF, we decided to add the primary topic and score results that included "off-topic" classifers such as "entertainment" and "social". This information could aid us in looking for alignment across scoring techniques.


In [17]:
# create new master df by adding topic and score columns to HF score df

# combinedf = summaries.join(newdf['topic1'])
# combinedf = combinedf.join(newdf['score1'])
# combinedf = combinedf.join(newdf['topic2'])
# combinedf = combinedf.join(newdf['score2'])

In [37]:
# combinedf

In [18]:
combinedf1 = summaries.join(dfClassifier['topic1'])
combinedf1 = combinedf1.join(dfClassifier['score1'])
combinedf1 = combinedf1.join(dfClassifier['topic2'])
combinedf1 = combinedf1.join(dfClassifier['score2'])

In [19]:
pd.set_option("max_colwidth", 30)
pd.set_option("expand_frame_repr", True)
combinedf1

,article,summary,hf1,hf2,hf3,hf4,hf5,hfAvg,kb1,kb2,kb3,kb4,kb5,kbAvg,topic1,score1,topic2,score2
0,https://datainnovation.org...,economist created visualiz...,0.5627,0.5556,0.5153,0.5132,0.4939,0.528140,0.408926,0.408615,0.395756,0.385963,0.365358,0.392923,economics,0.629220,politics,0.167246
1,https://venturebeat.com/ai...,among organization buildin...,0.9986,0.6797,0.6685,0.6572,0.6319,0.659325,0.515490,0.456996,0.455971,0.445295,0.442166,0.450107,technology,0.585379,innovation,0.180912
2,https://venturebeat.com/ai...,watch demand session today...,0.5767,0.5576,0.5529,0.5504,0.5455,0.551600,0.549421,0.546504,0.544978,0.534699,0.533749,0.541870,entertainment,0.176518,social,0.170545
3,https://venturebeat.com/ai...,generative going completel...,0.7521,0.7365,0.7272,0.7240,0.7216,0.732280,0.519292,0.496403,0.488997,0.482039,0.477190,0.486157,innovation,0.602014,business innovation,0.258873
4,https://www.businessinside...,personalized cancer vaccin...,0.5688,0.5227,0.4532,0.4527,0.4452,0.488520,0.428122,0.427320,0.422924,0.419798,0.395541,0.424541,innovation,0.430343,technology,0.140970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,https://www.cbsnews.com/ne...,spacex falcon rocket carry...,0.8183,0.4993,0.4775,0.4735,0.4719,0.480550,0.531412,0.511708,0.454913,0.436846,0.434590,0.473894,technology,0.496222,innovation,0.111382
121,https://www.cbsnews.com/vi...,rise could chatgpt take jo...,0.7697,0.7255,0.7166,0.7103,0.6910,0.710850,0.637201,0.625964,0.625526,0.619846,0.619757,0.622773,technology,0.675250,innovation,0.216986
122,https://www.cbsnews.com/ne...,jonas ohman founder ceo bl...,0.4923,0.4892,0.4892,0.4847,0.4596,0.488850,0.457436,0.457436,0.430608,0.427529,0.421351,0.438872,technology,0.349854,social,0.111065
123,https://www.cbsnews.com/ne...,metaverse asset digital ar...,0.6600,0.6304,0.6264,0.6229,0.6044,0.626567,0.614686,0.609527,0.597931,0.587996,0.571264,0.596281,games,0.463839,entertainment,0.240798


In [20]:
# write new combined df to JSON

combinedf1.to_json(r'/home/deleidos/Notebooks/JSONData/semanticSimArticlesTestingDF.json')
complete(state="updates written: ")


Cell updates written:  @ 2023-02-03 17:54:26


### Now let's look at and compare what would have been chosen if we used 'BoW' instead

In [20]:
def matchkeyword(keyword):
    if keyword in companies or keyword in technologies or keyword in generics or keyword in candidate_labels:
        return True

def load_candidate_articles_BoW(docs):
    articles_csv = []
    for doc in docs:
        url = doc[0]
#         print(f"URL: {url}")
        an_article = Article(url=url)
        an_article.download()
        an_article.throw_if_not_downloaded_verbose()
        an_article.parse()
        an_article.nlp()
#         print(f"Article URL: {url}, keywords : {an_article.keywords}")

        for keyword in an_article.keywords:
            if matchkeyword(keyword):
                articles_csv.append({'url': url,
                                    'text': an_article.summary, 
                                    'keywords': an_article.keywords,                                        
                }) 
    return  pd.DataFrame(articles_csv)


In [23]:
dfBoW = load_candidate_articles_BoW(docs)
complete(state="dfBoW")

/opt/conda/lib/python3.8/site-packages/requests/utils.py:488: DeprecationWarning: In requests 3.0, get_encodings_from_content will be removed. For more information, please see the discussion on issue #2266. (This warning should only appear once.)
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/requests/utils.py:488: DeprecationWarning: In requests 3.0, get_encodings_from_content will be removed. For more information, please see the discussion on issue #2266. (This warning should only appear once.)
  warnings.warn(



Cell dfBoW @ 2023-01-25 16:46:56


In [39]:
dfBoW

,url,text,keywords
0,https://datainnovation.org...,The Economist has created ...,"[republicans, twitter, mus..."
1,https://datainnovation.org...,The Economist has created ...,"[republicans, twitter, mus..."
2,https://datainnovation.org...,The Economist has created ...,"[republicans, twitter, mus..."
3,https://venturebeat.com/ai...,Among the organizations bu...,"[aws, sagemaker, cloud, ge..."
4,https://venturebeat.com/ai...,Among the organizations bu...,"[aws, sagemaker, cloud, ge..."
...,...,...,...
156,https://www.foxbusiness.co...,The outspoken and unapolog...,"[twitter, musk, company, d..."
157,https://www.foxbusiness.co...,The outspoken and unapolog...,"[twitter, musk, company, d..."
158,https://www.foxbusiness.co...,Massachusetts Institute of...,"[structures, creating, vox..."
159,https://www.foxbusiness.co...,Massachusetts Institute of...,"[structures, creating, vox..."


### Scoring from the Bag Of Words
###### Now, can we score (in some way) whether the associated keywords are indicative of innovation or technology?


## <font color='red'><B>This is left as an exercise for Anna</B></font> 


In [46]:
# kws = []

# for index, row in dfBoW.iterrows():
# #     kws.append([row['url'], row['keywords']])
# #     print(row['url'], row['keywords']) 
#     kws.append(row['keywords'])
# #     print(row['keywords'])

kws = dfBoW['keywords'].values.tolist()
tokens = [j for sub in kws for j in sub]
tokens = [t.split(',') for t in tokens]
# print(tokens)
# print(kws)

In [49]:
# Create dictionary from BoW
id2word = Dictionary(tokens)

# Create corpus
texts = tokens
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:10])

[[(0, 1)], [(1, 1)], [(2, 1)], [(3, 1)], [(4, 1)], [(5, 1)], [(6, 1)], [(7, 1)], [(8, 1)], [(9, 1)]]


In [51]:
# [[(id2word[i], freq) for i, freq in doc] for doc in corpus[:]]

In [52]:
lda_model = LdaModel(corpus=corpus,
                   id2word=id2word,
                   num_topics=10,
                   random_state=0,
                   chunksize=100,
                   alpha='auto',
                   per_word_topics=True)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.072*"intelligence" + 0.054*"technology" + 0.043*"cookies" + '
  '0.043*"experience" + 0.043*"site" + 0.042*"browser" + 0.041*"website" + '
  '0.033*"selection" + 0.033*"monitor" + 0.033*"websites"'),
 (1,
  '0.002*"platform" + 0.001*"generation" + 0.001*"wait" + 0.001*"future" + '
  '0.001*"robots" + 0.001*"builtin" + 0.001*"selfreplicating" + '
  '0.001*"company" + 0.001*"innovation" + 0.001*"defense"'),
 (2,
  '0.001*"risks" + 0.001*"visualizing" + 0.001*"builtin" + '
  '0.001*"selfreplicating" + 0.001*"robots" + 0.001*"images" + '
  '0.001*"stability" + 0.001*"changes" + 0.001*"company" + 0.001*"innovation"'),
 (3,
  '0.291*"power" + 0.058*"aws" + 0.001*"generative" + 0.001*"ceo" + '
  '0.001*"builtin" + 0.001*"selfreplicating" + 0.001*"robots" + '
  '0.001*"company" + 0.001*"innovation" + 0.001*"defense"'),
 (4,
  '0.011*"model" + 0.002*"watch" + 0.001*"images" + 0.001*"beat" + '
  '0.001*"twitter" + 0.001*"selfreplicating" + 0.001*"builtin" + '
  '0.001*"robots" + 0.001*

In [54]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
p = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
p 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.391032 -0.007515       1        1  38.378704
3      0.070011 -0.120439       2        1   7.656171
5      0.037868  0.010433       3        1   7.467054
6      0.049617  0.043669       4        1   7.296498
7      0.043272  0.021788       5        1   6.844248
4      0.038265  0.010737       6        1   6.782354
1      0.037282  0.009212       7        1   6.769149
8      0.039714  0.013636       8        1   6.471753
2      0.037476  0.009235       9        1   6.277465
9      0.037528  0.009243      10        1   6.056604, topic_info=             Term       Freq      Total Category  logprob  loglift
22          power  56.000000  56.000000  Default  30.0000  30.0000
32           work  25.000000  25.000000  Default  29.0000  29.0000
337  intelligence  69.000000  69.000000  Default  28.0000  28.0000
20             ai  12.000000  12.000000  Default  27.0000  27.0000
12            aws  12.000000  12.000000  Default  26.0000  26.0000
..            ...        ...        ...      ...      ...      ...
650      contract   0.192521   2.205724  Topic10  -6.6408   0.3654
42           help   0.192508  12.832523  Topic10  -6.6409  -1.3956
632       million   0.192447   2.142957  Topic10  -6.6412   0.3939
250      security   0.192425  10.463244  Topic10  -6.6413  -1.1919
85        digital   0.192406  11.652500  Topic10  -6.6414  -1.2996

[1500 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
20        5  0.891201               ai
308       1  0.903231           amazon
12        2  0.869871              aws
77        1  0.954507          browser
764       1  0.899030            build
765       1  0.419247          builtin
112       1  0.942017         business
14        1  0.933793            cloud
265       1  0.828865        companies
84        1  0.951246          cookies
758       1  0.898621         creating
114       1  0.907169             data
85        1  0.858185          digital
88        1  0.951953       experience
154       3  0.577225           health
42        1  0.857197             help
565       1  0.653997       innovation
337       1  0.970123     intelligence
762       1  0.898951           larger
760       1  0.898848              mit
18        6  0.560952            model
83        1  0.957691          monitor
2         1  0.855824             musk
65        1  0.936725      performance
3         3  0.423417         platform
22        2  0.962656            power
189       1  0.897062      researchers
763       1  0.898843            robot
767       1  0.419242           robots
90        1  0.949965             sale
250       1  0.860154         security
94        1  0.953212        selection
766       1  0.419248  selfreplicating
79        1  0.960971             site
757       1  0.899198       structures
761       1  0.898770           swarms
116       1  0.891840             tech
41        1  0.970958       technology
1         1  0.892674          twitter
759       1  0.899177           voxels
78        1  0.944431          website
81        1  0.928935         websites
32        4  0.935091             work
31        8  0.677495          working, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 6, 7, 8, 5, 2, 9, 3, 10])

### KeyBERT Estimation
Can we use KeyBERT to extract Keywords from the article and use these for scoring
To test this, please refer to [this notebook; BERT For Measuring Text Similarity](http://ec2-54-210-238-172.compute-1.amazonaws.com/notebooks/Notebooks/Newspaper/SemanticEstimation/KeyBERT_Similarity_Measure.ipynb)

In [55]:
kw_model = KeyBERT()
keywords = []
for doc in docs:
    keywords.append(
#       [
#            doc[0],
#           kw_model.extract_keywords(doc[1])
#       ]  
#     )
    kw_model.extract_keywords(doc[1]))

keywords

[[('twitter', 0.4421),
  ('musk', 0.4156),
  ('visualization', 0.3995),
  ('economist', 0.3951),
  ('politician', 0.3565)],
 [('cloud', 0.4524),
  ('fasterbuilding', 0.4193),
  ('generative', 0.3904),
  ('building', 0.3312),
  ('diffusion', 0.3285)],
 [('openai', 0.4455),
  ('demand', 0.4338),
  ('chatgpt', 0.3738),
  ('popularity', 0.3536),
  ('slowing', 0.3513)],
 [('generative', 0.5912),
  ('marketing', 0.4618),
  ('visual', 0.345),
  ('designer', 0.2949),
  ('writer', 0.2939)],
 [('vaccine', 0.461),
  ('cancer', 0.3531),
  ('personalized', 0.2992),
  ('cell', 0.2811),
  ('moderna', 0.2725)],
 [('starbase', 0.4675),
  ('facility', 0.4497),
  ('station', 0.4457),
  ('spacex', 0.4416),
  ('spot', 0.3887)],
 [('department', 0.4514),
  ('administration', 0.4275),
  ('acquisition', 0.3996),
  ('oversees', 0.3888),
  ('memorandum', 0.3766)],
 [('cooky', 0.3737),
  ('ccpa', 0.2966),
  ('information', 0.2641),
  ('data', 0.2593),
  ('toggle', 0.2476)],
 [('fcc', 0.4666),
  ('broadband', 0.4

In [61]:
keywords[0][0][0]

'twitter'

### Can we look at the relatedness of the terms in the keywords extracted to words indicating 'innovation' or 'technology'
Is this a viable approach?

## <font color='red'><B>This is left as an exercise for Anna</B></font> 


In [64]:
cols = list('abcde')
kwsDF = pd.DataFrame(keywords, columns=cols)
kwsDF

,a,b,c,d,e
0,"(twitter, 0.4421)","(musk, 0.4156)","(visualization, 0.3995)","(economist, 0.3951)","(politician, 0.3565)"
1,"(cloud, 0.4524)","(fasterbuilding, 0.4193)","(generative, 0.3904)","(building, 0.3312)","(diffusion, 0.3285)"
2,"(openai, 0.4455)","(demand, 0.4338)","(chatgpt, 0.3738)","(popularity, 0.3536)","(slowing, 0.3513)"
3,"(generative, 0.5912)","(marketing, 0.4618)","(visual, 0.345)","(designer, 0.2949)","(writer, 0.2939)"
4,"(vaccine, 0.461)","(cancer, 0.3531)","(personalized, 0.2992)","(cell, 0.2811)","(moderna, 0.2725)"
...,...,...,...,...,...
111,"(debt, 0.4109)","(loan, 0.3949)","(mortgage, 0.3787)","(refinancing, 0.3765)","(lender, 0.3762)"
112,"(varney, 0.4749)","(candidate, 0.4539)","(presidential, 0.4351)","(governor, 0.4343)","(stuart, 0.407)"
113,"(clown, 0.4538)","(murderer, 0.4082)","(killer, 0.3741)","(painting, 0.3655)","(wayne, 0.3391)"
114,"(housing, 0.3776)","(market, 0.3196)","(predicts, 0.3173)","(rise, 0.3047)","(unemployment, 0.2872)"


In [65]:
# demonstrate can flatten column and isolate score
dftest1 = pd.DataFrame(kwsDF['a'].values.tolist(), columns=['a','b'])
dftest1 = dftest1.drop(columns=['b'])
dftest1

,a
0,twitter
1,cloud
2,openai
3,generative
4,vaccine
...,...
111,debt
112,varney
113,clown
114,housing
